In [1]:
""" 3/27/18 
With a functional mister_ed package, it's time to run the madry challenge.

Step 1 is to build a defended network that can transfer to their tensorflow nets.
Because Madry uses tensorflow, transferability is a must. 

From Madry:
The model is a residual convolutional neural network consisting of five residual units and a fully connected layer. 
This architecture is derived from the "w32-10 wide" variant of the Tensorflow model repository. 
The network was trained against an iterative adversary that is allowed to perturb each pixel by at most epsilon=8.0.
"""
None

In [2]:
# Universal import block 
# Block to get the relative imports working 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch.nn as nn

import config
import prebuilt_loss_functions as plf
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain


In [3]:
# Step -1: load the classifier, data_loader, and the normalizer 
use_gpu = True
classifier_net = cifar_loader.load_pretrained_cifar_resnet(flavor=32,
                                                           use_gpu=use_gpu)
classifier_net.eval()

val_loader = cifar_loader.load_cifar_data('val', normalize=False, 
                                          use_gpu=use_gpu)

cifar_normer = utils.DifferentiableNormalize(mean=config.CIFAR10_MEANS,
                                             std=config.CIFAR10_STDS)


Files already downloaded and verified


In [4]:
# Step 0: initialize hyperparams for attack and training 
PGD_L_INF = 8.0 / 255.0
PGD_STEP_SIZE = 1.0 / 255.0
PGD_NUM_ITER = 16
PGD_TRAINING_PROP = 1.0

In [5]:
# Step 1: setup loss object 
standard_xentropy = plf.VanillaXentropy(classifier_net, normalizer=cifar_normer)

In [6]:
# Step 2: build attack object and its parameters
pgd_attack_object = aa.LInfPGD(classifier_net, cifar_normer, standard_xentropy)
pgd_attack_params = advtrain.AdversarialAttackParameters(
                            pgd_attack_object, 
                            PGD_TRAINING_PROP,
                            {'attack_kwargs': 
                                {'l_inf_bound': PGD_L_INF,
                                 'step_size': PGD_STEP_SIZE,
                                 'num_iterations': PGD_NUM_ITER,
                                 'random_init': True,
                                 'signed': True,
                                 'verbose': False}})


In [7]:
# Step 3: build training object, training loss, data loader

pgd_cifar_training = advtrain.AdversarialTraining(classifier_net, 
                                                  cifar_normer, 
                                                  'full_pgd_cifar_madry_params',
                                                  'cifar_resnet32')
train_loss = nn.CrossEntropyLoss()
train_loader = cifar_loader.load_cifar_data('train', normalize=False)



Files already downloaded and verified


In [14]:
pgd_attack_object.use_gpu

False

In [8]:
# Step 4: Do the training 
pgd_cifar_training.train(train_loader, 256, train_loss, 
                         attack_parameters=pgd_attack_params,
                         use_gpu=True, verbosity='medium')


<class 'torch.FloatTensor'>
<class 'torch.cuda.FloatTensor'>
WHAT?????


TypeError: add received an invalid combination of arguments - got (torch.cuda.FloatTensor), but expected one of:
 * (float value)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.cuda.FloatTensor[0m)
 * (torch.FloatTensor other)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.cuda.FloatTensor[0m)
 * (torch.SparseFloatTensor other)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.cuda.FloatTensor[0m)
 * (float value, torch.FloatTensor other)
 * (float value, torch.SparseFloatTensor other)
